training code: https://www.kaggle.com/hengzheng/nn-mlp-5folds-training

In [ ]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd

pd.set_option('max_columns', None)
from tqdm.notebook import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold

import joblib

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
class UBIQUANT_DATASET(Dataset):
    def __init__(self, df_data):
        self.ids = np.array(df_data['investment_id'].values.tolist(), dtype=np.int64)
        self.vals = np.array(df_data.iloc[:, 1:].values.tolist(), dtype=np.float64)
        self.len = df_data.shape[0]
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        ids_out = self.ids[index]
        vals_out = self.vals[index]
        return ids_out, vals_out

In [ ]:
def swish(x):
    return x * torch.sigmoid(x)


class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(3774, 64)
        self.emb_drop = nn.Dropout(0.1)
        
        self.bn1 = nn.BatchNorm1d(300)
        self.lin1 = nn.Linear(64+300, 32)
        self.lin2 = nn.Linear(32, 128)
        self.lin3 = nn.Linear(128, 64)
        self.lin4 = nn.Linear(64, 32)
        self.lin_drop = nn.Dropout(0.25)
        self.lin5 = nn.Linear(32, 1)    

    def forward(self, x_cat, x_cont):
        x1 = self.emb(x_cat)
        x1 = self.emb_drop(x1)
        
        x2 = self.bn1(x_cont)

        x = torch.cat([x1, x2], 1)
        x = swish(self.lin1(x))
        x = swish(self.lin2(x))
        x = swish(self.lin3(x))
        x = swish(self.lin4(x))
        x = self.lin5(x)
        
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
scaler = joblib.load('../input/ubiquant-simplemlp-5folds/minmaxscaler.pkl')
cont_feats = [f'f_{i}' for i in range(300)]

models_list = list()
for fold_id in range(5):
    model = SimpleMLP().to(device)
    model.load_state_dict(torch.load(f'../input/ubiquant-simplemlp-5folds/simple_mlp_model_{fold_id}.pth'))
    models_list.append(model)

In [ ]:
import ubiquant

env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(["row_id"], axis=1)
    test_df[cont_feats] = scaler.transform(test_df[cont_feats])
    test_set = UBIQUANT_DATASET(test_df)
    test_dataloader = DataLoader(test_set, batch_size=1024, 
                                 num_workers=2, pin_memory=True, shuffle=False)
    y_preds = list()
    with torch.no_grad():
        for i, (ids, vals) in enumerate(test_dataloader):
            ids = ids.to(device)
            vals = vals.to(device=device, dtype=torch.float)
            y_pred = np.zeros((len(ids), ))
            for model in models_list:
                model.eval()
                y_pred += model(ids, vals).detach().cpu().numpy().flatten() / 5
            y_preds.extend(y_pred)
    sample_prediction_df['target'] = y_preds
    env.predict(sample_prediction_df)
    display(sample_prediction_df)